### Dispatch model

In [1]:
! sudo apt update -y && sudo apt upgrade -y
! sudo apt install glpk-utils -y

! pip install pandas pyomo

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:2 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease [3632 B]
Hit:3 https://dl.yarnpkg.com/debian stable InRelease                           
Hit:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease   
Get:5 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main amd64 Packages [326 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease 
Get:7 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [30.9 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [4276 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [4186 kB]33m
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1297 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 

##### Importing packages

In [1]:
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
import time
import gc
import os


In [2]:
## To run the dispatcher later for multiple cases
## (Reference) Cases to run the dispatcher for after valdiation:
# power_PV=       [0.01, 0.03, 0.05, 0.1, 0.2]
# power_BESS=     [1, 3, 5, 10, 20]
# time_BESS=      [0, 1, 2, 3, 4]
# optimise_for=   ['Price', 'CO_2_eq']
# power_PV=       [0.01, 0.03, 0.1]
# power_BESS=     [1, 3, 10]
# time_BESS=      [0, 2, 4]
optimise_for=   ['price', 'co2']

# # Create a new list using list comprehension
# referenceCases = []
# for i in range(len(power_PV)):
#     for j in range(len(power_BESS)):
#         for k in range(len(time_BESS)):
#             for curve in optimise_for:
#                 referenceCases.append([power_PV[i], power_BESS[j], power_BESS[j] * time_BESS[k], curve])

#referenceCases

##### Call in the data, specify the frames of the problem (flows, hours)

In [4]:
price = pd.read_csv('Data/PriceCurve_SE4_2021.csv', sep = ';')
co2_pro = pd.read_csv('Data/production_emissions.csv')
co2_con = pd.read_csv('Data/consumption_emissions.csv')
pv = pd.read_csv('Data/oskarshamnpvprod.csv')
#load = pd.read_csv('Data/LoadCurveo.csv', sep = ',')
load = pd.read_csv('Data/LoadCurve_new.csv', sep=';')
load['Load'] = load['Load'].apply(lambda x: x.replace(',', '.')).astype(float)

data = load
data['Price'] = price['Grid_Price']
data['CO_2_eq'] = co2_pro['carbon_intensity_production_avg']
data['solar_PV'] = pv

#Converting from MW to kW
data['Load']= (data['Load'] * 1000)
#solar data is already in kW, but needs to be multiplied by the scaling factor (para el archivo original)
#para la nueva data se multiplica por 10 000
data['solar_PV']= (data['solar_PV'] * 10000)
data['Price']= (data['Price'] / 1000)
data['CO_2_eq']= (data['CO_2_eq'] / 1000)

data['Hour']= (data['Hour']).astype('int')

#data.head(48)
#data.head(10)

duplicated_rows = pd.concat([data.iloc[[-1]]] * 49, ignore_index=True)
#duplicated_rows

data = pd.concat([data, duplicated_rows], ignore_index=True)

#data.head(48)
#data.head(10)
#data

In [5]:
## Primary data parameters of our scenarios
pv_price= 80                #https://data.nrel.gov/submissions/53 in EUR/kW
bess_price= 150             #https://doi.org/10.1016/j.solener.2018.08.061 in EUR/kWh, adjusted for price decreases
# pv_opex= 17                 #EUR/kWh ->reference in excel
# bess_opex= 0.125            #EUR/kWh ->reference in excel
pv_opex= 3
bess_opex= 8

pv_co2= 33                  #kgCO2eq/kW_powerDC ->reference in excel
bess_co2= 80              #kgCO2eq/kWh_capacity ->reference in excel
pv_opex_co2= 0              #kgCO2eq/kW_powerDC ->assumption
bess_opex_co2= 2            #kgCO2eq/kW_powerDC ->assumption
discount_rate= 0.0485       #assumption
lifetime_project= 30        #for the project lifetime
lifetime_bess= 50            #for the BESS lifetime
degradation_rate= 0.015     #assumption (based on reaching 80% SoH in 8 years)

params = {
    'pv_price':         pv_price,
    'bess_price':       bess_price,
    'pv_opex':          pv_opex,
    'bess_opex':        bess_opex,
    'pv_co2':           pv_co2,
    'bess_co2':         bess_co2,
    'pv_opex_co2':      pv_opex_co2,
    'bess_opex_co2':    bess_opex_co2,
    'discount_rate':    discount_rate,
    'lifetime_project': lifetime_project,
    'lifetime_bess':    lifetime_bess,
    'degradation_rate': degradation_rate
}

In [6]:
## Define the sets/boundaries of the flow/hour table for our values
flows_global= [
    'P_PV_to_Load',
    'P_PV_to_BESS',
    'P_PV_curtailment',
    'P_PV_to_Grid',
    'P_BESS_to_Load',
    'P_BESS_to_Grid',
    'P_Grid_to_Load',
    'P_Grid_to_BESS',
]

hours_global=   set(range(8760 + 24))
time_window=    48
num_iterations= (8760/time_window)

## Grid availability for each hour is set to 1000 MW, should be more than enough for our model demand and any potential BESS demand for energy arbitrage
grid_production= 1000000


#type(float(data['Load'].max()))

In [7]:
## Ranges of cases for dispatch optimiser
## Total # of cases: 8 * 8 * 9 == 576
pv_load_multiples= [
    # 0,
    #0.5,
    1,
    # #1.5,
    # 2,
    #2.5,
    # 3,
    #3.5,
    # 4,
    # #4.5,
    5,
    # #5.5,
    # 6,
    #6.5,
    # 7,
    # #7.5,
    #8,
    # 10,
    # 13,
    15
]

bess_load_multiples= [
    #0,
    0.5,
    #  1,
    #  1.5,
    2,
    # #2.5,
    # 3,
    #3.5,
     4,
    # #4.5,
    #5,
    # #5.5,
    #6,
    # #6.5,
    # 7,
    #7.5,
    # 8,
    #10
]

bess_duration= [
    # 0,
    1,
    #2,
    # 3,
    4,
    # 5,
    #6,
    # 7,
    8
]

In [8]:
def file_exists(filename):
    try:
        # Try to open the file in read mode
        with open(filename, 'r'):
            pass
    except FileNotFoundError:
        return False
    return True

##### Dispatch Optimisation model

In [ ]:
## Comment this out (turn it off) if errors occur with pandas or with saved results, then troubleshoot
pd.options.mode.chained_assignment = None  # Default is 'warn'

flows_results = [
    'P_PV_to_Load',
    'P_PV_to_BESS',
    'P_PV_curtailment',
    'P_PV_to_Grid',
    'P_BESS_to_Load',
    'P_BESS_to_Grid',
    'P_Grid_to_Load',
    'P_Grid_to_BESS',
    'SoC'
]

## Other BESS parameters
efficiency_charge = 0.85
efficiency_discharge = 0.90
efficiency_inverter = 0.97

## what multiple of the load we will limit the grid injection by.
## Default is 20%
overreach_capacity = 20
demand_multiple = (float(data['Load'].max())) * (1 + overreach_capacity / 100)

for bess_cap in bess_duration:
    for bess_multiple in bess_load_multiples:
        for solar_multiple in pv_load_multiples:
            for curve in optimise_for:
                if not file_exists(f'Results/Batch5/CaseB/{curve}_{int(solar_multiple)}_{bess_multiple}_{bess_cap}.csv'):
                    results = pd.DataFrame(index=list(hours_global), columns=flows_results)

                    ## In kW, solar multiple multiplied by maxLoad (== 3000 kW) and divided by base PV production (100MW == 100 000 kW)
                    p_solar = (solar_multiple * 3000) / 100000
                    ## In kW, bess multiple multiplied by maxLoad (== 3000 kW)
                    p_bess = bess_multiple * 3000
                    ## In hours
                    t_bess = bess_cap
                    if t_bess != 0:
                        p_bess = bess_multiple * 3000  # Convert from MW to kW
                    else:
                        p_bess = 0
                    ## In kWh
                    e_bess = p_bess * t_bess
                    soc_initial = e_bess * 0.5
                    start_time = time.time()

                    for iteration in range(1, (int(num_iterations)) + 1):
                        hours = list(range(((iteration - 1) * time_window), ((iteration) * time_window)))

                        demand = {}
                        pv_production = {}
                        for hour in hours:
                            demand[hour] = data['Load'][hour]
                            pv_production[hour] = (data['solar_PV'][hour] * p_solar)

                        costs_keys = []
                        for flow in flows_global:
                            for hour in hours:
                                costs_keys.append((flow, hour))

                        costs_economic = {}
                        for i in range(len(costs_keys)):
                            if costs_keys[i][0] in ['P_Grid_to_Load', 'P_Grid_to_BESS']:
                                costs_economic[costs_keys[i]] = data['Price'][costs_keys[i][1]]
                            elif costs_keys[i][0] == 'P_PV_to_Grid':
                                costs_economic[costs_keys[i]] = (-1) * ((data['Price'][costs_keys[i][1]]) / 1000)
                            elif costs_keys[i][0] in ['P_PV_to_Load', 'P_PV_to_BESS']:
                                costs_economic[costs_keys[i]] = 0
                            elif costs_keys[i][0] == 'P_BESS_to_Grid':
                                costs_economic[costs_keys[i]] = (-1) * ((data['Price'][costs_keys[i][1]]) / 1000)
                            elif costs_keys[i][0] == 'P_BESS_to_Load':
                                costs_economic[costs_keys[i]] = 0
                            elif costs_keys[i][0] == 'P_PV_curtailment':
                                costs_economic[costs_keys[i]] = 1000 / 1000
                            else:
                                continue

                        costs_environmental = {}
                        for i in range(len(costs_keys)):
                            if costs_keys[i][0] in ['P_Grid_to_Load', 'P_Grid_to_BESS']:
                                costs_environmental[costs_keys[i]] = data['CO_2_eq'][costs_keys[i][1]]
                            elif costs_keys[i][0] == 'P_PV_to_Grid':
                                costs_environmental[costs_keys[i]] = (-1) * ((data['CO_2_eq'][costs_keys[i][1]]) / 1000)
                            elif costs_keys[i][0] in ['P_PV_to_Load', 'P_PV_to_BESS']:
                                costs_environmental[costs_keys[i]] = 0
                            elif costs_keys[i][0] == 'P_BESS_to_Grid':
                                costs_environmental[costs_keys[i]] = (-1) * ((data['CO_2_eq'][costs_keys[i][1]]) / 1000)
                            elif costs_keys[i][0] == 'P_BESS_to_Load':
                                costs_environmental[costs_keys[i]] = 0
                            elif costs_keys[i][0] == 'P_PV_curtailment':
                                costs_environmental[costs_keys[i]] = 1000 / 1000
                            else:
                                continue

                        ## Initialize model
                        model = pe.ConcreteModel()
                        ## Sets
                        model.flows = pe.Set(initialize=flows_global, ordered=True)
                        model.hours = pe.Set(initialize=hours, ordered=True)

                        ## Parameters
                        model.grid_production = pe.Param(initialize=grid_production)
                        model.demand = pe.Param(model.hours, initialize=demand)
                        model.pv_production = pe.Param(model.hours, initialize=pv_production)

                        model.Efficiency_charge = pe.Param(model.hours, initialize=efficiency_charge)
                        model.Efficiency_inverter = pe.Param(model.hours, initialize=efficiency_discharge)
                        model.Efficiency_discharge = pe.Param(model.hours, initialize=efficiency_inverter)

                        model.SoCmin = pe.Param(initialize=(e_bess * 0.05))
                        model.SoCmax = pe.Param(initialize=(e_bess * 0.95))

                        model.MaxCharge = pe.Param(initialize=p_bess)

                        ## OBS!: Read the last value of SoC, OR initial if first day
                        if iteration == 1:
                            model.SoCinitial = pe.Param(initialize=soc_initial)
                        else:
                            if pd.isna(results['SoC'].iloc[len(results) - 1]) == True:
                                model.SoCinitial = pe.Param(initialize=(e_bess * 0.1))
                            else:
                                last_SoC_value = float(results['SoC'].iloc[-1])
                                model.SoCinitial = pe.Param(initialize=last_SoC_value)

                        model.arbitrageLimit = pe.Param(model.hours, initialize=demand_multiple)
                        if curve == 'price':
                            model.costs = pe.Param(model.flows, model.hours, initialize=costs_economic, default=1)
                        elif curve == 'co2':
                            model.costs = pe.Param(model.flows, model.hours, initialize=costs_environmental, default=1)

                        ## Variables
                        model.p = pe.Var(model.flows, model.hours, domain=pe.NonNegativeReals)
                        model.SoC = pe.Var(model.hours, domain=pe.Reals, bounds=(model.SoCmin, model.SoCmax))
                        model.Sw_charge = pe.Var(model.hours, domain=pe.Binary)
                        model.Sw_discharge = pe.Var(model.hours, domain=pe.Binary)

                        ## Objective
                        minCosts = sum(model.p[f, t] * model.costs[f, t] for f in model.flows for t in model.hours)
                        model.objective = pe.Objective(sense=pe.minimize, expr=minCosts)

                        ## Constraints - Declaration
                        loadFullfilment = {t: (model.p['P_PV_to_Load', t] * efficiency_inverter + model.p['P_BESS_to_Load', t] * (efficiency_discharge * efficiency_inverter) + model.p['P_Grid_to_Load', t]) == model.demand[t] for t in model.hours}

                        pvProduction = {t: (model.p['P_PV_to_Load', t] + model.p['P_PV_to_BESS', t] + model.p['P_PV_to_Grid', t] + model.p['P_PV_curtailment', t]) == model.pv_production[t] for t in model.hours}

                        arbitrageFlow = {t: (model.p['P_BESS_to_Grid', t] * (efficiency_discharge * efficiency_inverter) + model.p['P_PV_to_Grid', t] * efficiency_inverter) <= model.arbitrageLimit[t] for t in model.hours}

                        gridFlow = {t: (model.p['P_Grid_to_Load', t] + model.p['P_Grid_to_BESS', t]) <= model.grid_production for t in model.hours}

                        bessChargeFlow = {t: (model.p['P_Grid_to_BESS', t] + model.p['P_PV_to_BESS', t]) <= model.Sw_charge[t] * model.MaxCharge for t in model.hours}
                        bessDischargeFlow = {t: (model.p['P_BESS_to_Grid', t] + model.p['P_BESS_to_Load', t]) <= model.Sw_discharge[t] * model.MaxCharge for t in model.hours}

                        limitValidation = {t: model.Sw_charge[t] + model.Sw_discharge[t] <= 1 for t in model.hours}

                        def storage_state(model, t):
                            if t == model.hours.first():
                                return model.SoC[t] == model.SoCinitial + ((model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * (efficiency_charge * efficiency_inverter)) - ((model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t]) / (efficiency_discharge * efficiency_inverter))
                            else:
                                return model.SoC[t] == model.SoC[t - 1] + ((model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * (efficiency_charge * efficiency_inverter)) - ((model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t]) / (efficiency_discharge * efficiency_inverter))

                        ## Constraints - Implementation
                        model.demandRule = pe.Constraint(model.hours, expr=loadFullfilment)
                        model.pvProductionRule = pe.Constraint(model.hours, expr=pvProduction)
                        model.arbitrageRule = pe.Constraint(model.hours, expr=arbitrageFlow)
                        model.gridRule = pe.Constraint(model.hours, expr=gridFlow)
                        model.chargeRule = pe.Constraint(model.hours, expr=bessChargeFlow)
                        model.dischargeRule = pe.Constraint(model.hours, expr=bessDischargeFlow)
                        model.limitValidation = pe.Constraint(model.hours, expr=limitValidation)
                        model.charge_state = pe.Constraint(model.hours, expr=storage_state)

                        ## Solve the model for the time window (iteration)
                        solver = po.SolverFactory('glpk')
                        solver.options['tmlim'] = 15
                        solver.options['mipgap'] = 0.05
                        modelresults = solver.solve(model, tee=False)

                        ## Results Handling
                        for flow in flows_results:
                            for hour in hours:
                                if flow == 'SoC':
                                    results['SoC'][hour] = model.SoC[hour].value
                                else:
                                    results[flow][hour] = model.p[flow, hour].value
                            results[flow] = results[flow].astype(float)

                        results['sum_power_flows'] = results.P_PV_to_Load + results.P_BESS_to_Load + results.P_Grid_to_Load
                        results['sum_power_flows'] = results['sum_power_flows'].astype(float)

                        for param in model.component_data_objects(pe.Param, active=True):
                            model.del_component(param)
                        for variable in model.component_data_objects(pe.Var, active=True):
                            model.del_component(variable)
                        for cons in model.component_data_objects(pe.Constraint, active=True):
                            model.del_component(cons)
                        for obj in model.component_data_objects(pe.Objective, active=True):
                            model.del_component(obj)
                        model.del_component(model.hours)
                        model.del_component(model.flows)

                        gc.collect()

                        # Save partial results and free memory
                        results_partial = results.loc[hours]
                        results_partial = results_partial.reset_index()
                        results_partial = results_partial.rename(columns={'index': 'Hour'})
                        partial_filename = f'Results/Batch5/CaseB/{curve}_{int(solar_multiple)}_{bess_multiple}_{t_bess}_partial_{iteration}.csv'
                        results_partial.to_csv(
                            partial_filename,
                            sep=',',
                            index=False,
                            columns=None
                        )
                        del results_partial
                        gc.collect()

                    # Unify partial results
                    partial_files = [f'Results/Batch5/CaseB/{curve}_{int(solar_multiple)}_{bess_multiple}_{t_bess}_partial_{i}.csv' for i in range(1, (int(num_iterations)) + 1)]
                    unified_results = pd.concat([pd.read_csv(f) for f in partial_files])
                    unified_results.to_csv(
                        f'Results/Batch5/CaseB/{curve}_{int(solar_multiple)}_{bess_multiple}_{t_bess}.csv',
                        sep=',',
                        index=False,
                        columns=None
                    )

                    # Remove partial files
                    for f in partial_files:
                        os.remove(f)

                    del unified_results
                    gc.collect()